In [3]:
import re
import pandas as pd
from datetime import datetime
from pathlib import Path
log_pattern = re.compile(
    r'(?P<host>\S+)\s+'              # host
    r'\S+\s+\S+\s+'                  # ignore ident, authuser
    r'\[(?P<time>.*?)\]\s+'          # timestamp
    r'"(?P<method>\S+)\s+'           # HTTP method
    r'(?P<url>\S+)\s*'               # URL
    r'\S*"\s+'                       # protocol
    r'(?P<status>\d{3})\s+'          # status code
    r'(?P<bytes>\S+)'                # bytes
)

In [10]:
!git clone https://github.com/myprivatelearningacc/Autoscaling-Analysis.git


Cloning into 'Autoscaling-Analysis'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), done.


In [11]:
%cd Autoscaling-Analysis

/content/Autoscaling-Analysis


In [12]:
!pip install gdown

In [14]:
!mkdir -p data/raw

In [15]:
!gdown 1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk -O data/raw/train.txt

Downloading...
From (original): https://drive.google.com/uc?id=1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk
From (redirected): https://drive.google.com/uc?id=1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk&confirm=t&uuid=96d80486-eec5-47b8-a543-3066a0c1e810
To: /content/Autoscaling-Analysis/data/raw/train.txt
100% 320M/320M [00:04<00:00, 73.4MB/s]


In [16]:
from pathlib import Path

DATA_PATH = Path("data/raw/train.txt")
print("File exists:", DATA_PATH.exists())
print("File size (MB):", DATA_PATH.stat().st_size / 1e6)


File exists: True
File size (MB): 319.688542


In [17]:
with open(DATA_PATH, "r", encoding="latin-1") as f:
    for _ in range(5):
        print(f.readline())


199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245

unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985

199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085

burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0

199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179



In [18]:
DATA_PATH = Path("data/raw/train.txt")
records = []

with open(DATA_PATH, "r", encoding="latin-1") as f:
    for line in f:
        match = log_pattern.search(line)
        if match:
            data = match.groupdict()

            # bytes: "-" → 0
            data["bytes"] = int(data["bytes"]) if data["bytes"].isdigit() else 0

            # timestamp → datetime
            data["time"] = datetime.strptime(
                data["time"], "%d/%b/%Y:%H:%M:%S %z"
            )

            records.append(data)

df = pd.DataFrame(records)
df.head()


,host,time,method,url,status,bytes
0,199.72.81.55,1995-07-01 00:00:01-04:00,GET,/history/apollo/,200,6245
1,unicomp6.unicomp.net,1995-07-01 00:00:06-04:00,GET,/shuttle/countdown/,200,3985
2,199.120.110.21,1995-07-01 00:00:09-04:00,GET,/shuttle/missions/sts-73/mission-sts-73.html,200,4085
3,burger.letters.com,1995-07-01 00:00:11-04:00,GET,/shuttle/countdown/liftoff.html,304,0
4,199.120.110.21,1995-07-01 00:00:11-04:00,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,200,4179


In [24]:
clean_path = "clean_parsed_log.csv"
df.to_csv(clean_path, index=False)
print("Saved to", clean_path)

Saved to clean_parsed_log.csv


In [22]:
from google.colab import files
files.download(clean_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>